# Computational Vision Project - Image Matching - Alignment

**Riccardo Caprile 4370774**

Starting from a set of local keypoints (SIFT), align image pairs or image sets with respect to a common reference frame.

Images of planar scenes can be registered by estimating an homography, that is a (projective) transformation between planes.

Test with images acquired by you of the same planar scene shot from different distances 


All the information about this Notebook are written in the report.

The output cells have been cleared otherwise the  file would have been too heavy for the upload to Aulaweb.

In [ ]:
import cv2 
import matplotlib.pyplot as plt
import numpy as np

MIN_MATCH_COUNT = 10

# Feature Matching Function

Section 3.3 of the Report

In [ ]:
def Feature_Matching(des1,des2,good_list,good,distance):
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1,des2,k=2)
       
    for m,n in matches:
        if m.distance < distance * n.distance:
            good_list.append([m])
            good.append(m)
    return good_list, good

 # Find Homography Function
 
 Section 3.4 of the Report

In [ ]:
def findHomography(good,kp_1,kp_2):
    if len(good)>MIN_MATCH_COUNT:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
        
    M, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC,5.0)
    return M

# Plotting Function

This a simple function that plot 2 different images in the same line of the output cell , with their own titles

In [ ]:
def Plotting_2_Images(img1,img2,img1_title,img2_title):
    fig = plt.figure(figsize = (30 ,50))
    fig.add_subplot(1,2,1)
    plt.title(img1_title)
    plt.imshow((cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)))
    fig.add_subplot(1,2,2)
    plt.title(img2_title)
    plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
    plt.show()

This function plot 3 different images in the same line of the output with their own titles

It also saves the result image in the project folder called Outputs

In [ ]:
def Plotting_3_Images(img1,img2,img3,img1_title,img2_title,img3_title):
    fig = plt.figure(figsize = (30 ,50))
    fig.add_subplot(1,3,1)
    plt.title(img1_title)
    plt.imshow((cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)))
    fig.add_subplot(1,3,2)
    plt.title(img2_title)
    plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
    fig.add_subplot(1,3,3)
    plt.title(img3_title)
    plt.imshow(cv2.cvtColor(img3, cv2.COLOR_BGR2RGB))
    cv2.imwrite("Images/Outputs/" + img3_title +".jpg", img3)
    plt.show()

# TEST : Painting rotated of 90 degrees

Section 4.1 of the Report

### Step 1 ---- > Read and Plot Images

In [ ]:
#Upload the images needed
Ref = cv2.imread("Images/Main_Frame.jpg")
test1 = cv2.rotate(Ref,cv2.ROTATE_90_CLOCKWISE)


Ref = cv2.cvtColor(Ref, cv2.COLOR_BGR2RGB)
test1 = cv2.cvtColor(test1, cv2.COLOR_BGR2RGB)

#Plot the images 
Plotting_2_Images(Ref, test1, "Reference Image", "Test image")

### Step 2 ---->  Keypoints Extraction

Section 3.1 and 3.2 of the report

In [ ]:
#SIFT Detector
sift = cv2.SIFT_create()

#Find keypoints
kp1, des1 = sift.detectAndCompute(Ref,None)
kp2, des2 = sift.detectAndCompute(test1,None)

sift_Ref = None
sift_test1 = None

In [ ]:
#Draw all the Keypoints found
sift_Ref = cv2.drawKeypoints(Ref, kp1, sift_Ref, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
sift_test1= cv2.drawKeypoints(test1, kp2, sift_test1, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

Plotting_2_Images(sift_Ref,sift_test1,"Reference Image Keypoints" , "Test Image Keypoints")

### Step 3 ----->  Feature Matching

Section 3.3 of the Report

In [ ]:
good = []
good_list = []

good_list , good = Feature_Matching (des1,des2,good_list,good,0.5)

In [ ]:
img_Matched = None
img_Matched = cv2.drawMatchesKnn(Ref,kp1,test1,kp2,good_list,img_Matched,flags=2)

plt.figure(figsize=(50, 100))
plt.imshow(cv2.cvtColor(img_Matched, cv2.COLOR_BGR2RGB))

### Step 4 ---- > Find Homography

Section 3.4 of the Report

In [ ]:
homography = findHomography(good, kp1, kp2)

### Step 5 -----> Warp Perspective

Section 3.5 of the Report

In [ ]:
aligned_img = cv2.warpPerspective(test1, homography, (Ref.shape[1], Ref.shape[0]))

In [ ]:
#Plot the result
Plotting_3_Images(Ref,test1,aligned_img, "Reference Image" , "Test Image",  "Aligned  Painting Rotated")

# Test : Painting taken from a different angle

Section 4.2 of the Report

In [ ]:
#Upload the images needed
Ref = cv2.imread("Images/Main_Frame.jpg")
test1 = cv2.imread("Images/Right_Frame.jpg")


Ref = cv2.cvtColor(Ref, cv2.COLOR_BGR2RGB)
test1 = cv2.cvtColor(test1, cv2.COLOR_BGR2RGB)

#Plot the images 
Plotting_2_Images(Ref, test1, "Referece Image", "Different Perspective Image")

### Step 2 ---->  Keypoints Extraction

In [ ]:
#SIFT Detector
sift = cv2.SIFT_create()

#Find keypoints
kp1, des1 = sift.detectAndCompute(Ref,None)
kp2, des2 = sift.detectAndCompute(test1,None)

sift_Ref = None
sift_test1 = None

In [ ]:
#Draw all the Keypoints found
sift_Ref = cv2.drawKeypoints(Ref, kp1, sift_Ref, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
sift_test1= cv2.drawKeypoints(test1, kp2, sift_test1, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

Plotting_2_Images(sift_Ref,sift_test1,"Reference Image Keypoints" , "Test Image Keypoints")

### Step 3 ----->  Feature Matching

In [ ]:
good = []
good_list = []

good_list , good = Feature_Matching (des1,des2,good_list,good,0.5)

In [ ]:
img_Matched = None
img_Matched = cv2.drawMatchesKnn(Ref,kp1,test1,kp2,good_list,img_Matched,flags=2)

plt.figure(figsize=(50, 100))
plt.imshow(cv2.cvtColor(img_Matched, cv2.COLOR_BGR2RGB))

### Step 4 ---- > Find Homography

In [ ]:
homography = findHomography(good, kp1, kp2)

### Step 5 -----> Warp Perspective

In [ ]:
aligned_img = cv2.warpPerspective(test1, homography, (Ref.shape[1], Ref.shape[0]))

In [ ]:
#Plot the result
Plotting_3_Images(Ref,test1,aligned_img, "Reference Image" , "Different Perspective Image",  "Aligned Painting  with Different Perspective")

# Test : Far Frame

Section 4.3 of the Report

In [ ]:
#Upload the images needed
Ref = cv2.imread("Images/Main_Frame.jpg")
test1 = cv2.imread("Images/Far_Frame.jpg")


Ref = cv2.cvtColor(Ref, cv2.COLOR_BGR2RGB)
test1 = cv2.cvtColor(test1, cv2.COLOR_BGR2RGB)

#Plot the images 
Plotting_2_Images(Ref, test1, "Reference Image", "Far Image")

### Step 2 ---->  Keypoints Extraction

In [ ]:
#SIFT Detector
sift = cv2.SIFT_create()

#Find keypoints
kp1, des1 = sift.detectAndCompute(Ref,None)
kp2, des2 = sift.detectAndCompute(test1,None)

sift_Ref = None
sift_test1 = None

In [ ]:
#Draw all the Keypoints found
sift_Ref = cv2.drawKeypoints(Ref, kp1, sift_Ref, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
sift_test1= cv2.drawKeypoints(test1, kp2, sift_test1, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

Plotting_2_Images(sift_Ref,sift_test1,"Reference Image Keypoints" , "Test Image Keypoints")

### Step 3 ----->  Feature Matching

In [ ]:
good = []
good_list = []

good_list , good = Feature_Matching (des1,des2,good_list,good,0.5)

In [ ]:
img_Matched = None
img_Matched = cv2.drawMatchesKnn(Ref,kp1,test1,kp2,good_list,img_Matched,flags=2)

plt.figure(figsize=(50, 100))
plt.imshow(cv2.cvtColor(img_Matched, cv2.COLOR_BGR2RGB))

### Step 4 ---- > Find Homography

In [ ]:
homography = findHomography(good, kp1, kp2)

### Step 5 -----> Warp Perspective

In [ ]:
aligned_img = cv2.warpPerspective(test1, homography, (Ref.shape[1], Ref.shape[0]))

In [ ]:
#Plot the result
Plotting_3_Images(Ref,test1,aligned_img, "Reference Image" , "Far Image",  "Aligned Far Painting")

# Test : 3D Dodecahedron

Section 4.4 of the Report

In [ ]:
#Upload the images needed
Ref = cv2.imread("Images/Ref_Prism.jpg")
test1 = cv2.imread("Images/Prism.jpg")


Ref = cv2.cvtColor(Ref, cv2.COLOR_BGR2RGB)
test1 = cv2.cvtColor(test1, cv2.COLOR_BGR2RGB)

#Plot the images 
Plotting_2_Images(Ref, test1, "Reference Image", "Test 3D Prism")

### Step 2 ---->  Keypoints Extraction

In [ ]:
#SIFT Detector
sift = cv2.SIFT_create()

#Find keypoints
kp1, des1 = sift.detectAndCompute(Ref,None)
kp2, des2 = sift.detectAndCompute(test1,None)

sift_Ref = None
sift_test1 = None

In [ ]:
#Draw all the Keypoints found
sift_Ref = cv2.drawKeypoints(Ref, kp1, sift_Ref, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
sift_test1= cv2.drawKeypoints(test1, kp2, sift_test1, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

Plotting_2_Images(sift_Ref,sift_test1,"Reference Image Keypoints" , "Test Image Keypoints")

### Step 3 ----->  Feature Matching

In [ ]:
good = []
good_list = []

good_list , good = Feature_Matching (des1,des2,good_list,good,0.75)

In [ ]:
img_Matched = None
img_Matched = cv2.drawMatchesKnn(Ref,kp1,test1,kp2,good_list,img_Matched,flags=2)

plt.figure(figsize=(50, 100))
plt.imshow(cv2.cvtColor(img_Matched, cv2.COLOR_BGR2RGB))

### Step 4 ---- > Find Homography

In [ ]:
homography = findHomography(good, kp1, kp2)

### Step 5 -----> Warp Perspective

In [ ]:
aligned_img = cv2.warpPerspective(test1, homography, (Ref.shape[1], Ref.shape[0]))

In [ ]:
#Plot the result
Plotting_3_Images(Ref,test1,aligned_img, "Reference Image" , "Test 3D Prism",  "Aligned 3D Prism")

# Test: 3D Space with Object and Drawings

Section 4.5 of the Report

In [ ]:
#Upload the images needed
Ref = cv2.imread("Images/Ref_Tape.jpg")
test1 = cv2.imread("Images/Tape.jpg")


Ref = cv2.cvtColor(Ref, cv2.COLOR_BGR2RGB)
test1 = cv2.cvtColor(test1, cv2.COLOR_BGR2RGB)

#Plot the images 
Plotting_2_Images(Ref, test1, "Reference Image", "3D Space with Tape and two Images")

### Step 2 ---->  Keypoints Extraction

In [ ]:
#SIFT Detector
sift = cv2.SIFT_create()

#Find keypoints
kp1, des1 = sift.detectAndCompute(Ref,None)
kp2, des2 = sift.detectAndCompute(test1,None)

sift_Ref = None
sift_test1 = None

In [ ]:
#Draw all the Keypoints found
sift_Ref = cv2.drawKeypoints(Ref, kp1, sift_Ref, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
sift_test1= cv2.drawKeypoints(test1, kp2, sift_test1, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

Plotting_2_Images(sift_Ref,sift_test1,"Reference Image Keypoints" , "Test Image Keypoints")

### Step 3 ----->  Feature Matching

In [ ]:
good = []
good_list = []

good_list , good = Feature_Matching (des1,des2,good_list,good,0.75)

In [ ]:
img_Matched = None
img_Matched = cv2.drawMatchesKnn(Ref,kp1,test1,kp2,good_list,img_Matched,flags=2)

plt.figure(figsize=(50, 100))
plt.imshow(cv2.cvtColor(img_Matched, cv2.COLOR_BGR2RGB))

### Step 4 ---- > Find Homography

In [ ]:
homography = findHomography(good, kp1, kp2)

### Step 5 -----> Warp Perspective

In [ ]:
aligned_img = cv2.warpPerspective(test1, homography, (Ref.shape[1], Ref.shape[0]))

In [ ]:
#Plot the result
Plotting_3_Images(Ref,test1,aligned_img, "Reference Image" , "3D Space with Tape and two Images",  "Aligned 3D Space with Tape and two Images")

# Test : Alignment of a form

Section 4.6 of the Report

In [ ]:
#Upload the images needed
Ref = cv2.imread("Images/Ref_Form.jpg")
test1 = cv2.imread("Images/Form.jpg")


Ref = cv2.cvtColor(Ref, cv2.COLOR_BGR2RGB)
test1 = cv2.cvtColor(test1, cv2.COLOR_BGR2RGB)

#Plot the images 
Plotting_2_Images(Ref, test1, "Reference Image", "Test Form")

### Step 2 ---->  Keypoints Extraction

In [ ]:
#SIFT Detector
sift = cv2.SIFT_create()

#Find keypoints
kp1, des1 = sift.detectAndCompute(Ref,None)
kp2, des2 = sift.detectAndCompute(test1,None)

sift_Ref = None
sift_test1 = None

In [ ]:
#Draw all the Keypoints found
sift_Ref = cv2.drawKeypoints(Ref, kp1, sift_Ref, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
sift_test1= cv2.drawKeypoints(test1, kp2, sift_test1, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

Plotting_2_Images(sift_Ref,sift_test1,"Reference Image Keypoints" , "Test Image Keypoints")

### Step 3 ----->  Feature Matching

In [ ]:
good = []
good_list = []

good_list , good = Feature_Matching (des1,des2,good_list,good,0.5)

In [ ]:
img_Matched = None
img_Matched = cv2.drawMatchesKnn(Ref,kp1,test1,kp2,good_list,img_Matched,flags=2)

plt.figure(figsize=(50, 100))
plt.imshow(cv2.cvtColor(img_Matched, cv2.COLOR_BGR2RGB))

### Step 4 ---- > Find Homography

In [ ]:
homography = findHomography(good, kp1, kp2)

### Step 5 -----> Warp Perspective

In [ ]:
aligned_img = cv2.warpPerspective(test1, homography, (Ref.shape[1], Ref.shape[0]))

In [ ]:
#Plot the result
Plotting_3_Images(Ref,test1,aligned_img, "Reference Image" , "Test Form",  "Aligned Test Form")

# Test : OCR

Section 4.7 of the Report

In [ ]:
#Upload the images needed
Ref = cv2.imread("Images/Ref_Book_OCR.jpg")
test1 = cv2.imread("Images/Book_OCR.jpg")


Ref = cv2.cvtColor(Ref, cv2.COLOR_BGR2GRAY)
test1 = cv2.cvtColor(test1, cv2.COLOR_BGR2GRAY)

#Plot the images 
Plotting_2_Images(Ref, test1, "Reference Image", "Test Book Cover")

### Step 2 ---->  Keypoints Extraction

In [ ]:
#SIFT Detector
sift = cv2.SIFT_create()

#Find keypoints
kp1, des1 = sift.detectAndCompute(Ref,None)
kp2, des2 = sift.detectAndCompute(test1,None)

sift_Ref = None
sift_test1 = None

In [ ]:
#Draw all the Keypoints found
sift_Ref = cv2.drawKeypoints(Ref, kp1, sift_Ref, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
sift_test1= cv2.drawKeypoints(test1, kp2, sift_test1, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

Plotting_2_Images(sift_Ref,sift_test1,"" , "")

### Step 3 ----->  Feature Matching

In [ ]:
good = []
good_list = []

good_list , good = Feature_Matching (des1,des2,good_list,good,0.5)

In [ ]:
img_Matched = None
img_Matched = cv2.drawMatchesKnn(Ref,kp1,test1,kp2,good_list,img_Matched,flags=2)

plt.figure(figsize=(50, 100))
plt.imshow(cv2.cvtColor(img_Matched, cv2.COLOR_BGR2RGB))

### Step 4 ---- > Find Homography

In [ ]:
homography = findHomography(good, kp1, kp2)

### Step 5 -----> Warp Perspective

In [ ]:
aligned_img = cv2.warpPerspective(test1, homography, (Ref.shape[1], Ref.shape[0]))

In [ ]:
#Plot the result
Plotting_3_Images(Ref,test1,aligned_img, "Reference Image" , "Test Book Cover",  "Aligned Test Book Cover")

# OCR Tests with two different Libraries

## Pytesseract 

Section 4.7.1 of the Report

In [ ]:
import pytesseract
from pytesseract import Output

In [ ]:
pytesseract.pytesseract.tesseract_cmd = 'D:\Program Files\Tesseract-OCR\\tesseract.exe'

In [ ]:
text = pytesseract.image_to_string(aligned_img,lang='eng')
print(text)

## Keras OCR 

Section 4.7.2 of the Report

In [ ]:
#!pip install -q keras-ocr

In [ ]:
import keras_ocr

In [ ]:
pipeline = keras_ocr.pipeline.Pipeline()

images = [keras_ocr.tools.read( "Images/Ref_Book_OCR.jpg")]
              
prediction_groups = pipeline.recognize(images)

predicted_image = prediction_groups[0]
for text,box in predicted_image:
    print(text)